# Agent Memory and Routing: Building a Multi-Talented Assistant

This notebook represents the culmination of our work in Agentic Architecture. We are moving beyond single-purpose agents and building a single, cohesive agent that possesses two critical, human-like abilities:

1.  **Memory:** The ability to remember past interactions and handle follow-up questions in a stateful conversation.
2.  **Routing:** The ability to intelligently choose the correct tool from a diverse toolkit based on the user's request.

In this lab, we will combine the state-of-the-art RAG pipeline we built in Module 1 with the tool-calling agent from Module 2. We will create a single `AgentExecutor` that is given two distinct tools:

-   A **RAG Tool** (`get_knowledge_from_library`): For answering questions about specific topics (the Transformer architecture).
-   A **Time Tool** (`get_current_time`): For answering questions about the current time.

By running a sequence of different queries, we will demonstrate that the agent can:
-   Correctly route a knowledge-based question to the RAG tool.
-   Correctly route a time-based question to the time tool.
-   Use its memory to answer a question about the conversation history without needing to use a tool at all.

This is the core engine of a truly useful "Auto-Support Agent" and demonstrates the power of modern, tool-calling LLMs to act as intelligent orchestrators.

In [ ]:
# First, install the necessary libraries
!pip install langchain langchain_openai chromadb pypdf langchain-community langchain-chroma

import os
from datetime import datetime
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain.agents import tool, create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferWindowMemory

# Set up your OpenAI API key
try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
except ImportError:
    print("Not in a Colab environment, assuming API key is set.")

# --- 1. Define the Tools ---
# Tool 1: The time tool from our previous lab
@tool
def get_current_time(tool_input: str = "") -> str:
    """
    Returns the current date and time as a string.
    Use this tool for any questions about the current time.
    """
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Tool 2: Our RAG retriever from Module 1
# We need to load the vector store we created in Lab 1.1
print("--- Loading RAG Retriever Tool ---")
persist_directory = './chroma_db_rag'
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
vector_store = Chroma(persist_directory=persist_directory, embedding_function=embedding_model)
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

@tool
def get_knowledge_from_library(query: str) -> str:
    """
    Your primary tool. Use this to find information to answer a user's question.
    This tool searches a library of documents about the Transformer architecture
    and attention mechanisms. Use it for any questions about these topics.
    """
    docs = retriever.get_relevant_documents(query)
    return "\n---\n".join([doc.page_content for doc in docs])

# Put all tools in a list
tools = [get_current_time, get_knowledge_from_library]

# --- 2. Create the Agent with Memory ---
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# The prompt now needs a placeholder for memory
# `MessagesPlaceholder` is a special class that handles inserting the chat history.
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# Define the memory
# We'll use a window memory to keep the last K turns of the conversation.
memory = ConversationBufferWindowMemory(
    k=5,
    memory_key="chat_history",
    return_messages=True
)

# Create the modern tool-calling agent
agent = create_openai_tools_agent(llm, tools, prompt)

# --- 3. Create the Agent Executor ---
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    memory=memory # IMPORTANT: Pass the memory to the executor
)

# --- 4. Run the Agent! ---
# First question (uses the RAG tool)
print("\n--- Query 1: RAG Tool ---")
response1 = agent_executor.invoke({"input": "What is the core idea of the attention mechanism?"})
print("\nFinal Answer:", response1['output'])

# Second question (uses the time tool)
print("\n--- Query 2: Time Tool ---")
response2 = agent_executor.invoke({"input": "Thanks! By the way, what time is it?"})
print("\nFinal Answer:", response2['output'])

# Third question (tests memory)
print("\n--- Query 3: Memory Test ---")
response3 = agent_executor.invoke({"input": "What was the first topic I asked you about?"})
print("\nFinal Answer:", response3['output'])